# Tool Calling in LangChain

* Tool Creation
* Tool Binding
    * Test Tool Binding Result
* Append Tool Result for LLM Execution

In [1]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

## **Step 1:** Tool Creation

In [3]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [4]:
# Tool Testing to see if it works
print(multiply.invoke({'a':3, 'b':4}))

12


In [ ]:
# Tool Info

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## **Step 2:** Tool Binding

In [7]:
# Create LLM Instance
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Bind the tool to the LLM
llm_with_tools = llm.bind_tools([multiply])

In [8]:
# Use "Messages" to use the LLM with tools

query = HumanMessage('Can you multiply 3 with 5?')
messages = [query]
print(messages)

[HumanMessage(content='Can you multiply 3 with 5?', additional_kwargs={}, response_metadata={})]


In [ ]:
result = llm_with_tools.invoke(messages)

# LLM will call "Tool" instead of giving us some text 
print(result)
messages.append(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_hJxkmKRQvAYKOJSeyiHha9rk', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BVJMaG8RKTp6MCGKKNat9rlQrIzKb', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-6e4a6564-3334-4419-a7b1-2a647f41fb90-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_hJxkmKRQvAYKOJSeyiHha9rk', 'type': 'tool_call'}] usage_metadata={'input_tokens': 62, 'output_tokens': 18, 'total_tokens': 80, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi

In [12]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 5},
 'id': 'call_hJxkmKRQvAYKOJSeyiHha9rk',
 'type': 'tool_call'}

In [ ]:
# Messages contains the tool call with its specific format
display(messages)

[HumanMessage(content='Can you multiply 3 with 5?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hJxkmKRQvAYKOJSeyiHha9rk', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BVJMaG8RKTp6MCGKKNat9rlQrIzKb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6e4a6564-3334-4419-a7b1-2a647f41fb90-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_hJxkmKRQvAYKOJSeyiHha9rk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_toke

## **Step 2.1 (Optional):** Tool Call Checking

In [14]:
# To check if our tool can actually run the instructions given out by LLM
tool_result = multiply.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='15', name='multiply', tool_call_id='call_hJxkmKRQvAYKOJSeyiHha9rk')

## **Step 3:** Append Tool Result for LLM Execution 

In [15]:
messages.append(tool_result)
messages

[HumanMessage(content='Can you multiply 3 with 5?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hJxkmKRQvAYKOJSeyiHha9rk', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BVJMaG8RKTp6MCGKKNat9rlQrIzKb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6e4a6564-3334-4419-a7b1-2a647f41fb90-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_hJxkmKRQvAYKOJSeyiHha9rk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_toke

In [16]:
llm_with_tools.invoke(messages).content

'The product of 3 and 5 is 15.'